In [117]:
import pandas as pd
import re
from collections import Counter

In [2]:
import spacy
nlp = spacy.load('en')

In [138]:
from gensim.models import Phrases
from gensim.models.phrases import Phraser

In [110]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [111]:
from nltk.probability import FreqDist #didn't end up using this

In [91]:
# Update puncuation list in spacy
nlp.vocab["$"].is_punct = True
nlp.vocab["|"].is_punct = True
nlp.vocab["+"].is_punct = True
nlp.vocab["<"].is_punct = True
nlp.vocab[">"].is_punct = True
nlp.vocab["="].is_punct = True
nlp.vocab["^"].is_punct = True
nlp.vocab["`"].is_punct = True
nlp.vocab["~"].is_punct = True

In [3]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 20)

In [4]:
clean_final = pd.read_csv("clean_final_news.csv", encoding = "utf8", index_col = 0)
clean_final.head()

,author,published,text,domain_rank,site_url,spam_score,title,response,length
0,SAM TANENHAUS,2016-11-20T03:51:00.000+02:00,Privacy Policy Eisenhowers two terms bore this...,98.0,nytimes.com,0.000,Opinion: Donald Trump’s Art of the New Deal?,Not fake,949
1,NaN,2016-11-20T00:52:00.000+02:00,Can Trump Save Their Jobs? by Nelson D. Sc...,98.0,nytimes.com,0.000,Carrier Workers for Trump,Not fake,372
2,Kathleen Elkins,2016-11-20T06:18:00.000+02:00,"Tuesday, 18 Oct 2016 | 10:25 AM ET CNBC.com Ed...",767.0,cnbc.com,0.008,Tennis star Caroline Wozniacki shares the mone...,Not fake,386
3,JEFF SOMMER,2016-11-20T00:04:00.000+02:00,Continue reading the main story Yet it is poss...,98.0,nytimes.com,0.000,Strategies: It’s Not Just the White House. Cha...,Not fake,1005
4,James Rufus Koren,2016-11-20T02:42:00.000+02:00,Wells Fargo hit with new sanctions following f...,609.0,latimes.com,0.264,Wells Fargo hit with new sanctions following f...,Not fake,809


In [102]:
# Remove hashtags from text
for idx, item in enumerate(clean_final.text):
    clean_final.text[idx] = re.sub(r'@([A-Za-z0-9_]+)', "", str(item))
    
    if idx%500 == 0:
        print ('Here is the ',idx,'th item')

C:\Users\jgoldste\Anaconda2\envs\py35\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Here is the  0 th item
Here is the  500 th item
Here is the  1000 th item
Here is the  1500 th item
Here is the  2000 th item
Here is the  2500 th item
Here is the  3000 th item
Here is the  3500 th item
Here is the  4000 th item
Here is the  4500 th item
Here is the  5000 th item
Here is the  5500 th item
Here is the  6000 th item
Here is the  6500 th item
Here is the  7000 th item
Here is the  7500 th item
Here is the  8000 th item
Here is the  8500 th item
Here is the  9000 th item
Here is the  9500 th item
Here is the  10000 th item
Here is the  10500 th item
Here is the  11000 th item
Here is the  11500 th item
Here is the  12000 th item
Here is the  12500 th item
Here is the  13000 th item
Here is the  13500 th item
Here is the  14000 th item
Here is the  14500 th item
Here is the  15000 th item
Here is the  15500 th item
Here is the  16000 th item
Here is the  16500 th item
Here is the  17000 th item
Here is the  17500 th item
Here is the  18000 th item
Here is the  18500 th ite

In [103]:
# Convert text and response to array 
x_text = clean_final.text.values
y_response = clean_final.response.values

In [104]:
# helper function to eliminate tokens that are pure punctuation, whitespace, or stopword
# can be updated based on desired filtering 

def process_txt(token):
    return token.is_punct or token.is_space or token.is_stop or token.like_num

In [119]:
# function to take array of articles and turn them into nested list of tokens
def lemmatize_txt(array):
    lemma = []
    
    for doc in nlp.pipe(array, batch_size=50,
                        n_threads=-1):
        if doc.is_parsed:
            lemma.append([n.lemma_ for n in doc if not process_txt(n)])
        
        else:
            lemma.append(None)
    
    return lemma

In [106]:
# function to recombine nested list of tokens into full articles 
def lemma_combine(lis):
    parsed_articles = []
    
    for i in range(len(lis)):
        concat_art = ' '.join(lis[i])
        parsed_articles.append(concat_art)
    
    return parsed_articles

In [107]:
# function to match "cleaned" text back up with response variable
def zip_response(observations, response):
    response = response.tolist()
    
    return list(zip(observations, response))

In [148]:
# Test pipeline
lem = lemmatize_txt(x_text[5:10])
lem_comb = lemma_combine(lem)
zip_response(lem_comb,y_response[5:10])

[("privacy policy liberal society depend illiberal pre liberal substructure answer varied human need meaning belong vertical dimension human life hope mortality john stuart mill karl marx adequately address american history substructure take form bond family life power usually protestant religion flag wave patriotism anglo saxon culture immigrant expect assimilate foundation manifest illiberalisms evil religious intolerance racism chauvinism oppression private domestic power provide moral cultural metaphysical common ground political reformer abolitionist social gospellers new dealers civil right marcher rely expand liberalism promise post-1960s liberal politic contrast experiment cut western society loose foundation set tune john lennons imagine heaven religion country border parochial loyalty kind value center leave far leave alike neoliberal hop manage global capitalism neo marxists hop transcend unfortunately value imagine simply sufficient need human life people desire solidarity 

In [147]:
# Model to create bi-grams for our text - would do this before lemma_combine() function
phrases = Phrases(lem)
bigram = Phraser(phrases)
bigram_lem = list(bigram[lem])

[['breaking',
  'putin',
  'gave',
  'obama',
  'hours',
  'ships',
  'open',
  'fire',
  'oct',
  'previous',
  'post',
  'russia',
  'united',
  'states',
  'ally',
  'easily',
  'remember',
  'tense',
  'long',
  'last',
  'cold',
  'war',
  'leave',
  'russia',
  'angry',
  'suffer',
  'demoralizing',
  'defeat',
  'world',
  'power',
  'united',
  'states',
  'vladimir',
  'putins',
  'good',
  'friend',
  'realize',
  'russia',
  'super',
  'power',
  'world',
  'seriously',
  'hillary',
  'obama',
  'dislike',
  'putin',
  'answer',
  'strength',
  'toughness',
  'exactly',
  'evident',
  'like',
  'donald',
  'trump',
  'way',
  'project',
  'strength',
  'obama',
  'hillary',
  'continue',
  'putin',
  'book',
  'blame',
  'cyber',
  'attack',
  'wikileaks',
  'friday',
  'october',
  'putin',
  'issue',
  'global',
  'message',
  'hillary',
  'clinton',
  'barrack',
  'obama',
  'continue',
  'slander',
  'russia',
  'go',
  'shoot',
  'u.s.',
  'jet',
  'entire',
  'article'

In [125]:
# Preparing text in case we want to filter out words with low frequencies
# Flatten list of lemmas to use in counter below
full_lem = [item for sublist in lem for item in sublist]

In [128]:
# Get frequency count of all words in corpus
counts = Counter()
for lemma in full_lem:
    counts[lemma] += 1

In [130]:
# Pull out words with fewer than five occurences 
{k:v for k,v in counts.items() if v < 5}

{'10-year': 1,
 '27-year': 1,
 'accomplish': 1,
 'accord': 2,
 'accuse': 2,
 'act': 1,
 'ad': 1,
 'afar': 1,
 'air': 1,
 'allegation': 1,
 'allegiance': 1,
 'ally': 2,
 'america': 3,
 'american': 2,
 'americas': 1,
 'amount': 1,
 'anally': 1,
 'angry': 1,
 'anonymous': 1,
 'answer': 1,
 'anti': 1,
 'appeasement': 1,
 'area': 1,
 'army': 1,
 'arrive': 1,
 'aspect': 1,
 'assert': 1,
 'assistance': 1,
 'asylum': 3,
 'august': 1,
 'authority': 1,
 'baby': 2,
 'baltic': 3,
 'barbaric': 1,
 'barrack': 1,
 'beer': 1,
 'belong': 1,
 'besiege': 1,
 'big': 1,
 'blame': 1,
 'boil': 1,
 'bomb': 1,
 'book': 1,
 'borders': 1,
 'boy': 2,
 'breaking': 1,
 'british': 1,
 'busy': 2,
 'campaign': 1,
 'captive': 1,
 'carry': 3,
 'cause': 1,
 'center': 1,
 'centre': 2,
 'charge': 1,
 'child': 3,
 'christian': 1,
 'citizen': 1,
 'citizenry': 1,
 'city': 2,
 'citys': 1,
 'claim': 2,
 'click': 3,
 'clinton': 2,
 'cnn': 1,
 'cold': 2,
 'collaborate': 2,
 'come': 1,
 'commander': 1,
 'compare': 1,
 'compound': 